## Chemical Compounds Join Analysis

We want to know which compounds are in common between two alternative reporting standards and which ones belong to one one of each. We also want to know the different names in the different standards even if they have the same CAS number

Tutorial by Rafael Guerra

#### Import libraries

In [34]:
import pandas as pd
import numpy as np

#### Load Data

There are two reporting standards for chemicals, the SmartLabel standard and the CPISI standard.

In [35]:
sl = pd.read_csv('smart_label_data.csv')
cpisi = pd.read_csv('cpisi.csv')

#### Examine Data

We have a few issues we need to take care of such as a value of 0, not available, and NaN. There are also values with the / or - signs and after consulting with specialists in the field, these are all acceptable formats. This may only be present in one of the reporting standards, but it is good practice to check this issue for both datasets.

In [36]:
sl.head(10)

,ingredient,cas_number
0,BUTYLATED HYDROXYTOLUENE (BHT),0
1,BUTYLPHENYL METHYLPROPIONAL,0
2,C.I. 21108,0
3,C.I. 50415B,0
4,C.I. 77266,0
5,C.I. ACID BLUE 182,72152-54-6
6,C.I. ACID BLUE 7,3486-30-4
7,C.I. ACID BLUE 80,4474-24-2
8,C.I. ACID YELLOW 17,6359-98-4
9,C.I. DIRECT BLUE 199,12222-04-7


In [37]:
cpisi.head(10)

,ingredient,cas_number
0,"(3R,4S,5S,6R)-2-HEXOXY-6-(HYDROXYMETHYL)OXANE-...",54549-24-5
1,(4-FORMYLPHENYL)BORONIC ACID,87199-17-5
2,"[(2R,3R,4R,5R)-2-(2-AMINO-6-OXO-3H-PURIN-9-YL)...",12237-24-0
3,[3-[[(2R)-1-[(2R)-1-[(2S)-2-[(3R)-3-HYDROXYBUT...,68400-67-9
4,"1,2-THIAZOL-3-ONE",1003-07-2
5,"1,3,5-TRIAZINE-2,4,6(1H,3H,5H)-TRIONE, 1,3,5-T...",87-90-1
6,"1,3-CYCLOHEXANEDIMETHANAMINE",2579-20-6
7,"1,4-BIS(2,4,6-TRIMETHYLANILINO)ANTHRACENE-9,10...",116-75-6
8,"1,4-DIAMINO-2,3-DICHLOROANTHRACENE-9,10-DIONE",81-42-5
9,"1-[3-(2,4-DIAMINO-6-METHYLQUINAZOLIN-7-YL)PHEN...",93348-22-2


#### Remove NaN, Not Available, and values of 0

Since we want to retain all original data when we report individual items, I suggest an approach to create an additional feature instead of mutating the cas_number feature. This feature will be called mutated_cas and will converge the value of NaN, 0, and Not Available to the string 'SL Only' or 'CPISI Only'

In [38]:
sl['mutated_cas'] = sl['cas_number'].replace(['0','Not Available','Propietary',np.nan,'NA'],'SL Only')

In [39]:
cpisi['mutated_cas'] = cpisi['cas_number'].replace(['0','Not Available','Propietary',np.nan,'NA'],'CPSI Only')

In [40]:
sl.head()

,ingredient,cas_number,mutated_cas
0,BUTYLATED HYDROXYTOLUENE (BHT),0,SL Only
1,BUTYLPHENYL METHYLPROPIONAL,0,SL Only
2,C.I. 21108,0,SL Only
3,C.I. 50415B,0,SL Only
4,C.I. 77266,0,SL Only


In [41]:
cpisi.head()

,ingredient,cas_number,mutated_cas
0,"(3R,4S,5S,6R)-2-HEXOXY-6-(HYDROXYMETHYL)OXANE-...",54549-24-5,54549-24-5
1,(4-FORMYLPHENYL)BORONIC ACID,87199-17-5,87199-17-5
2,"[(2R,3R,4R,5R)-2-(2-AMINO-6-OXO-3H-PURIN-9-YL)...",12237-24-0,12237-24-0
3,[3-[[(2R)-1-[(2R)-1-[(2S)-2-[(3R)-3-HYDROXYBUT...,68400-67-9,68400-67-9
4,"1,2-THIAZOL-3-ONE",1003-07-2,1003-07-2


#### Using Merge to create common dataframe

First, we generate common_items.csv which will take items of two databases (FL and CPISI) that have common mutated_cas values.

In [42]:
common_items = pd.merge(sl,cpisi,left_on='mutated_cas',right_on='mutated_cas',suffixes=('_sl', '_cpisi'))

In [43]:
common_items

,ingredient_sl,cas_number_sl,mutated_cas,ingredient_cpisi,cas_number_cpisi
0,C.I. ACID BLUE 7,3486-30-4,3486-30-4,CI 42080,3486-30-4
1,C.I. ACID BLUE 80,4474-24-2,4474-24-2,CI 61585,4474-24-2
2,C.I. ACID YELLOW 17,6359-98-4,6359-98-4,CI 18965,6359-98-4
3,C.I. DIRECT BLUE 199,12222-04-7,12222-04-7,C.I. DIRECT BLUE 199,12222-04-7
4,C.I. DIRECT BLUE 86,1330-38-7,1330-38-7,CI 74180,1330-38-7
...,...,...,...,...,...
119,ISOPROPANOLDL,67-63-0,67-63-0,ISOPROPYL ALCOHOL,67-63-0
120,ISOTRIDECANOL ETHOXYLATED,9043-30-5,9043-30-5,ISOTRIDECANOL ETHOXYLATED 9EO,9043-30-5
121,LACTIC ACID,79-33-4,79-33-4,LACTIC ACID,79-33-4
122,LACTIC ACID (ÁCIDO LÁCTICO),79-33-4,79-33-4,LACTIC ACID,79-33-4


As we can see, there is too much overlapping information, so we will go ahead and select only the features we are interested in, which is the common CAS number, and then each database's name for that ingredient

In [44]:
common_items = common_items[['ingredient_sl','ingredient_cpisi','mutated_cas']]

In [45]:
common_items.head()

,ingredient_sl,ingredient_cpisi,mutated_cas
0,C.I. ACID BLUE 7,CI 42080,3486-30-4
1,C.I. ACID BLUE 80,CI 61585,4474-24-2
2,C.I. ACID YELLOW 17,CI 18965,6359-98-4
3,C.I. DIRECT BLUE 199,C.I. DIRECT BLUE 199,12222-04-7
4,C.I. DIRECT BLUE 86,CI 74180,1330-38-7


And now we are ready to save this as a CSV. We include index = False so that we don't have a column with enumeration.

In [46]:
common_items.to_csv('common_items.csv',index=False)

#### Create SL Only CSV

For this, once again we will use the merge function in pandas with the mutated_cas, but we will only keep the items that are present in the SL dataset. We will be using the tilde in Pandas which is the BITWISE operator. Basically, it returns the boolean opposite. So, in this case, instead of returning _merge == 'both', it's actually everything except for that

In [47]:
outer_join_sl = pd.merge(sl,common_items,on='mutated_cas',how='outer', indicator = True)

In [48]:
anti_join_sl = outer_join_sl[~(outer_join_sl._merge == 'both')].drop('_merge', axis = 1)

In [49]:
anti_join_sl

,ingredient,cas_number,mutated_cas,ingredient_sl,ingredient_cpisi
0,BUTYLATED HYDROXYTOLUENE (BHT),0,SL Only,NaN,NaN
1,BUTYLPHENYL METHYLPROPIONAL,0,SL Only,NaN,NaN
2,C.I. 21108,0,SL Only,NaN,NaN
3,C.I. 50415B,0,SL Only,NaN,NaN
4,C.I. 77266,0,SL Only,NaN,NaN
...,...,...,...,...,...
360,FRAGRANCE,100-52-7,100-52-7,NaN,NaN
363,GLYCERINE,8013-25-0,8013-25-0,NaN,NaN
373,HYDROGEN SULFATE,14996-02-2,14996-02-2,NaN,NaN
378,ISOPENTANE,78-78-4,78-78-4,NaN,NaN


In [50]:
print("Length of SL: ", len(sl))
print("Length of Common Items: ", len(common_items))
print("Length of SL Only: ", len(anti_join_sl))
print("Sum of SL + Common Items: ", len(anti_join_sl)+len(common_items))
print("Error: ", len(sl)-(len(anti_join_sl)+len(common_items)))

Length of SL:  310
Length of Common Items:  124
Length of SL Only:  186
Sum of SL + Common Items:  310
Error:  0


As we can see everything adds up -- the common items + the SL only equals the SL length. We're ready to export SL only items as 'sl_only.csv'

In [51]:
sl_only = anti_join_fl[['ingredient','cas_number']]

In [52]:
sl_only.to_csv('sl_only.csv',index=False)

#### Create CPISI Only CSV

Same thing as above, so I will skip the textual explanations

In [53]:
outer_join_cpisi = pd.merge(cpisi,common_items,on='mutated_cas',how='outer', indicator = True)

In [54]:
anti_join_cpisi = outer_join_cpisi[~(outer_join_cpisi._merge == 'both')].drop('_merge', axis = 1)

In [55]:
print("Length of CPISI: ", len(cpisi))
print("Length of Common Items: ", len(common_items))
print("Length of CPISI Only: ", len(anti_join_cpisi))
print("Sum of CPISI + Common Items: ", len(anti_join_cpisi)+len(common_items))
print("Error: ", len(cpisi)-(len(anti_join_cpisi)+len(common_items)))

Length of CPISI:  911
Length of Common Items:  124
Length of CPISI Only:  815
Sum of CPISI + Common Items:  939
Error:  -28


UH-OH! Something is off. There is a difference of 28 items from the original CPISI dataset and the reconstructed one. As it turns out, the reason is that one compound present in the CPISI database may have matched more than one compound in the SL database. So, there are duplicate entries. So let's try again by filtering for unique entries.

In [56]:
common_items.head()

,ingredient_sl,ingredient_cpisi,mutated_cas
0,C.I. ACID BLUE 7,CI 42080,3486-30-4
1,C.I. ACID BLUE 80,CI 61585,4474-24-2
2,C.I. ACID YELLOW 17,CI 18965,6359-98-4
3,C.I. DIRECT BLUE 199,C.I. DIRECT BLUE 199,12222-04-7
4,C.I. DIRECT BLUE 86,CI 74180,1330-38-7


In [57]:
common_items_unique = common_items[['ingredient_cpisi','mutated_cas']]

In [58]:
common_items_unique = common_items_unique.drop_duplicates()

In [59]:
common_items_unique.head()

,ingredient_cpisi,mutated_cas
0,CI 42080,3486-30-4
1,CI 61585,4474-24-2
2,CI 18965,6359-98-4
3,C.I. DIRECT BLUE 199,12222-04-7
4,CI 74180,1330-38-7


In [60]:
outer_join_cpisi = pd.merge(cpisi,common_items_unique,on='mutated_cas',how='outer', indicator = True)
anti_join_cpisi = outer_join_cpisi[~(outer_join_cpisi._merge == 'both')].drop('_merge', axis = 1)

In [61]:
print("Length of CPISI: ", len(cpisi))
print("Length of Common Items: ", len(common_items_unique))
print("Length of CPISI Only: ", len(anti_join_cpisi))
print("Sum of CPISI + Common Items: ", len(anti_join_cpisi)+len(common_items_unique))
print("Error: ", len(cpisi)-(len(anti_join_cpisi)+len(common_items_unique)))

Length of CPISI:  911
Length of Common Items:  96
Length of CPISI Only:  815
Sum of CPISI + Common Items:  911
Error:  0


Voila! Now we're ready to export it

In [62]:
cpisi_only = anti_join_cpisi[['ingredient','cas_number']]

In [63]:
cpisi_only.to_csv('cpisi_only.csv',index=False)